In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
train_dir = 'D:\DEPI\Project\DEPI_proj_dataset'
test_dir = 'D:\DEPI\Project\Test_set'


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up ImageDataGenerators
train_data = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_data = ImageDataGenerator(rescale=1./255)

train_gen = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

val_gen = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

test_gen = test_data.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)


Found 23274 images belonging to 37 classes.
Found 5807 images belonging to 37 classes.
Found 1027 images belonging to 37 classes.


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Sequential

# Define the model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Input shape
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_gen.num_classes, activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# import tensorflow as tf

# # Define the model checkpoint callback
# checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
#     'model_weights.weights.h5',  # Correct file extension
#     save_weights_only=True,
#     save_best_only=True,
#     monitor='val_loss',
#     mode='min',
#     verbose=1
# )



import os
import tensorflow as tf

# Path to save weights
#weights_dir = '/content/drive/MyDrive/Sign Language Recognition_DEPI/weights'
weights_path = 'model_weights.h5'

# Define the model checkpoint callback to save weights
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    weights_path,  # Full path to save the weights
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)


In [ ]:
from PIL import Image, UnidentifiedImageError
import os

# Function to check for corrupted images
def is_image_corrupted(img_path):
    try:
        img = Image.open(img_path)
        img.verify()  # Verify if it's an image
        return False
    except (UnidentifiedImageError, IOError):
        print(f"Corrupted image: {img_path}")
        return True

# Function to remove corrupted images from the dataset
def remove_corrupted_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            img_path = os.path.join(root, file)
            if is_image_corrupted(img_path):
                os.remove(img_path)
                print(f"Removed {img_path}")

# Clean your train, validation, and test directories
remove_corrupted_images(train_dir)
remove_corrupted_images(val_dir)
remove_corrupted_images(test_dir)

# Now proceed with the ImageDataGenerator
train_data = ImageDataGenerator(rescale=1./255)
val_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)

train_gen = train_data.flow_from_directory(train_dir, target_size=(224, 224), batch_size=64, class_mode='categorical')
val_gen = val_data.flow_from_directory(val_dir, target_size=(224, 224), batch_size=64, class_mode='categorical')
test_gen = test_data.flow_from_directory(test_dir, target_size=(224, 224), batch_size=64, class_mode='categorical')


KeyboardInterrupt: 

In [6]:
from tensorflow.keras.callbacks import EarlyStopping
history = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    callbacks=[checkpoint_cb, EarlyStopping(monitor='val_loss', patience=5)]
)


Epoch 1/50
364/364 [==============================] - ETA: 0s - loss: 6.9661 - accuracy: 0.1792
Epoch 1: val_loss improved from inf to 43.10599, saving model to model_weights.h5
364/364 [==============================] - 1353s 4s/step - loss: 6.9661 - accuracy: 0.1792 - val_loss: 43.1060 - val_accuracy: 0.0334
Epoch 2/50
364/364 [==============================] - ETA: 0s - loss: 1.8937 - accuracy: 0.4344
Epoch 2: val_loss improved from 43.10599 to 7.43188, saving model to model_weights.h5
364/364 [==============================] - 168s 462ms/step - loss: 1.8937 - accuracy: 0.4344 - val_loss: 7.4319 - val_accuracy: 0.2463
Epoch 3/50
364/364 [==============================] - ETA: 0s - loss: 1.2193 - accuracy: 0.6273
Epoch 3: val_loss improved from 7.43188 to 4.54672, saving model to model_weights.h5
364/364 [==============================] - 1163s 3s/step - loss: 1.2193 - accuracy: 0.6273 - val_loss: 4.5467 - val_accuracy: 0.4376
Epoch 4/50
364/364 [==============================] - ETA

In [ ]:
model.save('final_testing.h5')

In [3]:
import tensorflow as tf
import cv2
import numpy as np

model = tf.keras.models.load_model('final_testing.h5')
image = cv2.imread('D:/DEPI/Project/Train/e/6.jpg')
image1 = cv2.resize(image, (224,224))
print(image1.shape)
prediction = model.predict(np.expand_dims(image1, axis=0))
prediction

(224, 224, 3)
1/1 [==============================] - 23s 23s/step


array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 9.465985e-35,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00]], dtype=float32)

In [5]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
#model = tf.keras.models.load_model('final_testing.h5')

# Define the image path
image_path = 'D:\DEPI\Project\DEPI_proj_dataset/a/6.jpg'

# Load the image and resize it to the required input size (224x224 in your case)
image = cv2.imread(image_path)
image_resized = cv2.resize(image, (224, 224))

# Normalize the image (assuming your ImageDataGenerator applied normalization)
image_resized = image_resized / 255.0

# Expand dimensions to match the input shape of the model (1, 224, 224, 3)
image_expanded = np.expand_dims(image_resized, axis=0)

# Make prediction
prediction = model.predict(image_expanded)

# Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]

# If you used ImageDataGenerator for training, you can access the class indices like this:
# Assuming you used ImageDataGenerator for training and it had a `class_indices` attribute
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('D:/DEPI/Project/Train/', target_size=(224, 224))

# Mapping the class index back to the class label
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Reverse the dictionary
predicted_label = class_labels[predicted_class_index]

# Display the predicted label
print(f"Predicted Label: {predicted_label}")


1/1 [==============================] - 0s 22ms/step
Found 23264 images belonging to 37 classes.
Predicted Label: 10


In [ ]:
# model.load_weights('/content/drive/MyDrive/Sign Language Recognition_DEPI/weights/model_weights.weights.h5')


# Load the model weights with mismatched layers skipped
# Load the model weights with mismatched layers skipped
model.load_weights('/content/drive/MyDrive/Sign Language Recognition_DEPI/weights/model_weights.weights.h5',
                   skip_mismatch=True)



/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:454: UserWarning: A total of 7 objects could not be loaded. Example error message for object <Conv2D name=conv2d, built=True>:

Layer 'conv2d' expected 2 variables, but received 0 variables during loading. Expected: ['kernel', 'bias']

List of objects that could not be loaded:
[<Conv2D name=conv2d, built=True>, <BatchNormalization name=batch_normalization, built=True>, <Conv2D name=conv2d_1, built=True>, <BatchNormalization name=batch_normalization_1, built=True>, <Conv2D name=conv2d_2, built=True>, <BatchNormalization name=batch_normalization_2, built=True>, <Dense name=dense, built=True>]
  warnings.warn(msg)


In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_gen)
print(f'Test accuracy: {test_acc:.4f}')
print(f'Test loss: {test_loss:.4f}')


17/17 [==============================] - 31s 2s/step - loss: 0.2801 - accuracy: 0.9503
Test accuracy: 0.9503
Test loss: 0.2801


In [ ]:
import os
print(os.listdir('.'))  # Lists all files in the current directory


['.config', 'drive', 'sample_data']


In [ ]:
import os

# Path to save weights
weights_dir = '/content/drive/MyDrive/Sign Language Recognition_DEPI/weights'

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for a few epochs to verify
model.fit(train_generator, epochs=2, validation_data=validation_generator)


NameError: name 'train_generator' is not defined